In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import TimestampType
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

22/08/13 20:11:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/13 20:11:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/13 20:11:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/13 20:11:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/08/13 20:11:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/08/13 20:11:06 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [3]:
# 3
olist_order_items_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_order_items_dataset.csv")
# 8
olist_sellers_dataset = spark.read.options(header='True', inferSchema='True', delimiter=',') \
                            .csv("data/olist_sellers_dataset.csv")

In [5]:
olist_order_items_dataset.show()
olist_sellers_dataset.show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35|  58.9|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13| 239.9|        19.93|
|000229ec398224ef6...|            1|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-18 14:48:30| 199.0|        17.87|
|00024acbcdf0a6daa...|            1|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-15 10:10:18| 12.99|        12.79|
|00042b26cf59d7ce6...|            1|ac6c3623068f30de0...|df560393f3a51e745...|2017-02-13 13:57:51| 199.9|        18.14|
|00048cc3ae777c65d...|            1|ef92

In [7]:
items = olist_order_items_dataset.select(col("seller_id").alias("seller_id_2"),"price")
sellers = olist_sellers_dataset.select("seller_id","seller_city","seller_state")
join_table = items.join(sellers, items.seller_id_2 == sellers.seller_id, "Inner")
join_table = join_table.drop("seller_id_2")
join_table.show()

+------+--------------------+--------------------+------------+
| price|           seller_id|         seller_city|seller_state|
+------+--------------------+--------------------+------------+
|  58.9|48436dade18ac8b2b...|       volta redonda|          SP|
| 239.9|dd7ddc04e1b6c2c61...|           sao paulo|          SP|
| 199.0|5b51032eddd242adc...|       borda da mata|          MG|
| 12.99|9d7a1d34a50524090...|              franca|          SP|
| 199.9|df560393f3a51e745...|              loanda|          PR|
|  21.9|6426d21aca402a131...|      ribeirao preto|          SP|
|  19.9|7040e82f899a04d1b...|           sao paulo|          SP|
| 810.0|5996cddab893a4652...| presidente prudente|          SP|
|145.95|a416b6a846a117243...|           sao paulo|          SP|
| 53.99|ba143b05f0110f0dc...|           sao paulo|          SP|
| 59.99|cc419e0650a3c5ba7...|         santo andre|          SP|
|  45.0|8602a61d680a10a82...|           sao paulo|          SP|
|  74.0|4a3ca9315b744ce9f...|           

In [10]:
join_table.createOrReplaceTempView("join_table")

#### Order by seller_state

In [27]:
df = spark.sql("""WITH seller AS (
               SELECT
               seller_id,
               seller_state,
               ROUND(SUM(price)) AS sum_price
               FROM join_table
               GROUP BY seller_state, seller_id
               )
               SELECT seller_id, seller_state, sum_price
               FROM (SELECT *,
               ROW_NUMBER() OVER(PARTITION BY seller_state ORDER BY sum_price DESC) AS row_number
               FROM seller)
               WHERE row_number < 3
               """)
df.show()

+--------------------+------------+---------+
|           seller_id|seller_state|sum_price|
+--------------------+------------+---------+
|04308b1ee57b6625f...|          SC|  60131.0|
|eeb6de78f79159600...|          SC|  43740.0|
|3364a91ec4d56c98e...|          RO|   3580.0|
|a5259c149128e82c9...|          RO|   1182.0|
|47efca563408aae19...|          PI|   2522.0|
|327b89b872c14d1c0...|          AM|   1177.0|
|9d5a9018aee56acb3...|          GO|  10292.0|
|750303a20e9c56b2a...|          GO|   8085.0|
|2dee2ce60de9709b1...|          MT|   7785.0|
|abcd2cb37d46c2c8f...|          MT|   4880.0|
|4869f7a5dfa277a7d...|          SP| 229473.0|
|4a3ca9315b744ce9f...|          SP| 200473.0|
|001cca7ae9ae17fb1...|          ES|  25080.0|
|33dd941c27854f762...|          ES|   6788.0|
|a6bd7d1ccdac48c6b...|          PB|  14369.0|
|07017df32dc5f2f1d...|          PB|    946.0|
|87142160b41353c4e...|          RS|  31096.0|
|b32be1695eb7ec5f1...|          RS|  26874.0|
|b1fecf4da1fa2689b...|          MS

#### Order by seller_city

In [28]:
df = spark.sql("""WITH seller AS (
               SELECT
               seller_id,
               seller_city,
               ROUND(SUM(price)) AS sum_price
               FROM join_table
               GROUP BY seller_city, seller_id
               )
               SELECT seller_id, seller_city, sum_price
               FROM (SELECT *,
               ROW_NUMBER() OVER(PARTITION BY seller_city ORDER BY sum_price DESC) AS row_number
               FROM seller)
               WHERE row_number < 3
               """)
df.show()

+--------------------+--------------------+---------+
|           seller_id|         seller_city|sum_price|
+--------------------+--------------------+---------+
|da20530872245d6cd...|           igrejinha|    315.0|
|c33847515fa6305ce...|             brusque|  15520.0|
|ad97a199236354e53...|             brusque|  13206.0|
|2c4c47cb51acd5ea5...|            buritama|   2576.0|
|f181738b150df1f37...|         carapicuiba|   5530.0|
|f680f85bee2d25355...|         carapicuiba|   5184.0|
|60da8bfa7eebe230b...|    fernando prestes|     87.0|
|527801b552d0077ff...|               garca|  17943.0|
|c12b92bf1c350f3e6...|               garca|   4809.0|
|e333046ce6517bd8b...|             ipaussu|   7268.0|
|3c03b12bab54d8b37...|  sao joao de meriti|   4236.0|
|117cfc326c6d50da6...|  sao joao de meriti|    455.0|
|45213867cefbf2cd4...|              araras|   4955.0|
|31da954dc0855f249...|              araras|   4583.0|
|7a241947449cc45db...|           jacutinga|   8028.0|
|198c7ea11960a9844...|      

22/08/14 13:15:44 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 5692441 ms exceeds timeout 120000 ms
22/08/14 13:15:44 WARN SparkContext: Killing executors is not supported by current scheduler.
